# Bellabeat Smart Device Usage Analysis
### Case Study 2: How Can a Wellness Technology Company Play It Smart?

**Author:** Danijia Haggins  
**Date:** October 2025  
**Tools:** Python, Pandas, Matplotlib, NumPy

## **Scenario** 
I am a consumer insights analyst on the marketing analytics team at Bellabeat, a tech company that makes wellness products for women. The CCO of the company believes that analyzing smart device fitness data could provide valuable insights to inform Bellbeats marketing strategy. 

## **Business Task**:
Analyze smart device usage from FitBit users to discover trends in activity, sleep, and wellness habits. Apply these insights to help **Bellabeat** understand how consumers enage wit health-tracking devices. Apply these insights to help Bellabeat improve marketing strategies for the Leaf wellness tracker. 

[Data source](https://www.kaggle.com/datasets/arashnic/fitbit)

In this case study I am following 6 steps of the data analysis process: 
1. Ask
2. Prepare
3. Process
4. Analyze
5. Share
6. Act 


## ASK
**Key stakeholders**:
1. Urška Sršen (Cofounder & Chief Creative Officer)
2. Sando Mur (Cofounder, Executive Team)

**Key Questions Guiding Analysis**
1. What are the main trends in smart device usage?
2. How do these trends relate to Bellabeat customers?
3. How can these insights inform Bellabeats marketing strategy?

## 1. Prepare

### About this data
* The data is being loaded, processed, and analyzed within Kaggle notebooks using python, stored for the purposes of this project memory
* The time range this data represents is March 12, 2016-May 12, 2016
* The source data is stored in CSVs (29 total)
* The data is mostly long format, with a few of the csvs replicated in wide format
* The data has been made available by Creative Commons under the [CCO: Public Domain License](https://creativecommons.org/publicdomain/zero/1.0/)

In [1]:
# installing libraries needed 

import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb


## 2. Process 
* I'm using python to process (clean/transform) because:
    * I can clean, transform, and analyze the data in one place without switching platforms like with sql and excel.
    * Python is also good for scalability as it can support high-volume data better than spreadsheets can (excel won't display more than 1,048,576 rows--this data includes more rows than that).
    * I can document every step and analyze in one place, creating easily reproducible results.

### 📈🛑 Data Limitations
* There are two folders containing data, one folder for user data between March 12, 2016 to April 11, 2016 and another for data from April 12, 2016 to May 12, 2016. Each folder contains 11 csvs with the same columns, and will need to be combined in the data cleaning/preprocessing.
* Two participants (IDs 2891001357 and 6391747486) appear only in the first dailyActivity_merged dataset (from 3.12.16 to 4.11.16) and have no recorded activity after April 11, 1016. This may indicate device non-use or drop out during the study period. 
* Some csvs only represent a subset of data for users between April 12, 2016 - May 12, 2016: dailySteps, dailyintensities, dailycalories, and sleepday. This data is also redundant as total counts for steps and calories are included in the dailyActivities csvs, and steps are included in the minuteSteps csvs. We will not be using these csvs for the purposes of this analysis.
* Some of the minute data is too granular for the purposes of this analysis: minuteCalories, minuteIntensities, and minuteMETS will not be used for the purposes of this analysis.
*  Users may not wear their devices every day

### 📁 Data Preparation 
Of the 29 csvs provided in the fitbit dataset, only the following were used for this analysis: 
* daily_activity
* heartrate_seconds
* minute_sleep
* hourly_calories
* hourly_intensities
* hourly_steps
* weight_logInfo 

These were selected because they contain the most relevant and complete data for understanding user activity and health behavior. Files with incomplete, duplicate, or overly granular data (such as minute level logs) were excluded to simplify analysis and maintain clarity. 

### 🔄 Data Loading
* Import the csvs into dataframes 
* Inspect structure
* Combine csvs with matching columns + data into one dataframe
* Check dataframe data types to avoid computational errors. 

In [2]:
# loading the csv data into separate dataframes
da1 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/dailyActivity_merged_1.csv')
da2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/dailyActivity_merged_2.csv')
hs1 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/heartrate_seconds_merged_1.csv')
hs2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/heartrate_seconds_merged_2.csv')
hc1 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/hourlyCalories_merged_1.csv')
hc2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/hourlyCalories_merged_2.csv')
hi1 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/hourlyIntensities_merged_1.csv')
hi2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/hourlyIntensities_merged_2.csv')
hstps1 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/hourlySteps_merged_1.csv')
hstps2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/hourlySteps_merged_2.csv')
msl1= pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/minuteSleep_merged_1.csv')
msl2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/minuteSleep_merged_2.csv')
wlg1 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_3.12.16_to_4.11.16/weightLogInfo_merged_1.csv') 
wlg2 = pd.read_csv('/kaggle/input/bella-beat-case-study/data/fitabase_data_4.12.16_to_5.12.16/weightLogInfo_merged_2.csv')



In [3]:
# inspecting dataframes structure

# store data frames loaded above in a list 
dfs = [da1, da2, hs1, hs2, hc1, hc2, hi1, hi2, hstps1, hstps2, msl1, msl2, wlg1, wlg2]

# iterate over the list with a loop
# enumerate(iterable, start=0)
for i, x in enumerate(dfs, 1): # i = counter, start=1, x = df from dfs list
    print(f"Dataframe {i}:") # print number of the dataframe
    display(x.head(2)) # use display() for notebook friendly display
    print("\n") # add space between outputs

Dataframe 1:


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154




Dataframe 2:


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797




Dataframe 3:


...

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,4/5/2016 11:59:59 PM,53.299999,117.506384,22.0,22.969999,True,1459900799000
1,1927972279,4/10/2016 6:33:26 PM,129.600006,285.719105,NaN,46.169998,False,1460313206000




Dataframe 14:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.65,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.65,True,1462319999000


In [4]:
# check for nulls / missing data
# create a dictionary with the dataframes and their names
dict1 = {'da1': da1, 
       'da2': da2, 
       'hs1': hs1, 
       'hs2': hs2, 
       'hc1': hc1, 
       'hc2': hc2, 
       'hi1': hi1, 
       'hi2': hi2, 
       'hstps1': hstps1, 
       'hstps': hstps2,  
       'msl1': msl1, 
       'msl2': msl2, 
       'wlg1': wlg1, 
       'wlg2': wlg2
      }

# iterate over dict1
for name, df in dict1.items():
    print(f"Null values in {name}:")
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0: # can use .any() or .sum()
        print(null_counts[null_counts > 0]) # Print only columns with nulls
    else:
        print("No null values found.")
    print("\n")



Null values in da1:
No null values found.


Null values in da2:
No null values found.


Null values in hs1:
No null values found.


Null values in hs2:
No null values found.


Null values in hc1:
No null values found.


Null values in hc2:
No null values found.


Null values in hi1:
No null values found.


Null values in hi2:
No null values found.


Null values in hstps1:
No null values found.


Null values in hstps:
No null values found.


Null values in msl1:
No null values found.


Null values in msl2:
No null values found.


Null values in wlg1:
Fat    31
dtype: int64


Null values in wlg2:
Fat    65
dtype: int64




**Note:** 
Datframes wlg1 & wlg2 contain NaN values in the body fat column because some users did not record their body fat percentages. This will not cause errors in analysis. Leaving this as is but making a note here. 

In [5]:
# combining csv's with the same columns into one dataframe
daily_activity = pd.concat([da1, da2])
heartrate_seconds = pd.concat([hs1, hs2])
hourly_calories = pd.concat([hc1, hc2])
hourly_intensities = pd.concat([hi1, hi2])
hourly_steps = pd.concat([hstps1, hstps2])
minute_sleep = pd.concat([msl1, msl2])
weight_log_info = pd.concat([wlg1, wlg2])

In [6]:
# store combined + other dataframes in a dataframe
all_dfs = [daily_activity, heartrate_seconds, hourly_calories, hourly_intensities, hourly_steps, minute_sleep, weight_log_info]

# view first 5 rows of each dataframe
for i, x in enumerate(all_dfs, 1): 
    print(f'Dataframe {i}:')
    display(x.head())
    print('\n')

Dataframe 1:


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154
2,1503960366,3/27/2016,12736,8.53,8.53,0.0,4.66,0.16,3.71,0.0,56,5,268,605,1944
3,1503960366,3/28/2016,13231,8.93,8.93,0.0,3.19,0.79,4.95,0.0,39,20,224,1080,1932
4,1503960366,3/29/2016,12041,7.85,7.85,0.0,2.16,1.09,4.61,0.0,28,28,243,763,1886




Dataframe 2:


,Id,Time,Value
0,2022484408,4/1/2016 7:54:00 AM,93
1,2022484408,4/1/2016 7:54:05 AM,91
2,2022484408,4/1/2016 7:54:10 AM,96
3,2022484408,4/1/2016 7:54:15 AM,98
4,2022484408,4/1/2016 7:54:20 AM,100




Dataframe 3:


...



Dataframe 6:


,Id,date,value,logId
0,1503960366,3/13/2016 2:39:30 AM,1,11114919637
1,1503960366,3/13/2016 2:40:30 AM,1,11114919637
2,1503960366,3/13/2016 2:41:30 AM,1,11114919637
3,1503960366,3/13/2016 2:42:30 AM,1,11114919637
4,1503960366,3/13/2016 2:43:30 AM,1,11114919637




Dataframe 7:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,4/5/2016 11:59:59 PM,53.299999,117.506384,22.0,22.969999,True,1459900799000
1,1927972279,4/10/2016 6:33:26 PM,129.600006,285.719105,NaN,46.169998,False,1460313206000
2,2347167796,4/3/2016 11:59:59 PM,63.400002,139.773078,10.0,24.770000,True,1459727999000
3,2873212765,4/6/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1459987199000
4,2873212765,4/7/2016 11:59:59 PM,57.200001,126.104416,NaN,21.650000,True,1460073599000


In [7]:
# checking data types
all_dfs[0].info() # daily_activity, position 0 in all_dfs 
all_dfs[1].info() # heartrate seconds, position 1 in all_dfs
all_dfs[2].info() # hourly calories, position 2 in all_dfs
all_dfs[3].info() # hourly intensities, position 3 in all_dfs
all_dfs[4].info() # hourly steps, position 4 in all_dfs
all_dfs[5].info() # minute sleep, position 5 in all_dfs
all_dfs[6].info() # weight_log_info, position 6 in all_dfs

<class 'pandas.core.frame.DataFrame'>
Index: 1397 entries, 0 to 939
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        1397 non-null   int64  
 1   ActivityDate              1397 non-null   object 
 2   TotalSteps                1397 non-null   int64  
 3   TotalDistance             1397 non-null   float64
 4   TrackerDistance           1397 non-null   float64
 5   LoggedActivitiesDistance  1397 non-null   float64
 6   VeryActiveDistance        1397 non-null   float64
 7   ModeratelyActiveDistance  1397 non-null   float64
 8   LightActiveDistance       1397 non-null   float64
 9   SedentaryActiveDistance   1397 non-null   float64
 10  VeryActiveMinutes         1397 non-null   int64  
 11  FairlyActiveMinutes       1397 non-null   int64  
 12  LightlyActiveMinutes      1397 non-null   int64  
 13  SedentaryMinutes          1397 non-null   int64  
 14  Calories      

In [8]:
# convert datetime columns from objects to datetime64 datatypes & check the columns and datatypes
daily_activity['ActivityDate'] = pd.to_datetime(daily_activity['ActivityDate'])
heartrate_seconds['Time'] = pd.to_datetime(heartrate_seconds['Time'])
hourly_calories['ActivityHour'] = pd.to_datetime(hourly_calories['ActivityHour'])
hourly_intensities['ActivityHour'] = pd.to_datetime(hourly_intensities['ActivityHour'])
hourly_steps['ActivityHour'] = pd.to_datetime(hourly_steps['ActivityHour'])
minute_sleep['date'] = pd.to_datetime(minute_sleep['date'])
weight_log_info['Date'] = pd.to_datetime(weight_log_info['Date'])
 

/tmp/ipykernel_13/1549329201.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hourly_calories['ActivityHour'] = pd.to_datetime(hourly_calories['ActivityHour'])
/tmp/ipykernel_13/1549329201.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hourly_intensities['ActivityHour'] = pd.to_datetime(hourly_intensities['ActivityHour'])
/tmp/ipykernel_13/1549329201.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hourly_steps['ActivityHour'] = pd.to_datetime(hourly_steps['ActivityHour'])
/tmp/ipykernel_13/1549329201.py:8: UserWarning: Could not infer format, so each element wil

In [9]:
# check that datetime columns changed to datetime64
daily_activity['ActivityDate'].info() # daily_activity still an object but thats fine i think
heartrate_seconds['Time'].info() # heartrate seconds
hourly_calories['ActivityHour'].info() # hourly calories
hourly_intensities['ActivityHour'].info() # hourly intensities
hourly_steps['ActivityHour'] .info() # hourly steps
minute_sleep['date'].info() # minute sleep
weight_log_info['Date'].info() # weight_log_info

<class 'pandas.core.series.Series'>
Index: 1397 entries, 0 to 939
Series name: ActivityDate
Non-Null Count  Dtype         
--------------  -----         
1397 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 21.8 KB
<class 'pandas.core.series.Series'>
Index: 3638339 entries, 0 to 2483657
Series name: Time
Non-Null Count    Dtype         
--------------    -----         
3638339 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 55.5 MB
<class 'pandas.core.series.Series'>
Index: 46183 entries, 0 to 22098
Series name: ActivityHour
Non-Null Count  Dtype         
--------------  -----         
46183 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 721.6 KB
<class 'pandas.core.series.Series'>
Index: 46183 entries, 0 to 22098
Series name: ActivityHour
Non-Null Count  Dtype         
--------------  -----         
46183 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 721.6 KB
<class 'pandas.core.series.Series'>
Index: 46183 entrie

In [10]:
# checking dates: 

# view first 5 rows of each dataframe
for i, x in enumerate(all_dfs, 1): 
    print(f'Dataframe {i}:')
    display(x.head())
    print('\n')



Dataframe 1:


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,2016-03-25,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,2016-03-26,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154
2,1503960366,2016-03-27,12736,8.53,8.53,0.0,4.66,0.16,3.71,0.0,56,5,268,605,1944
3,1503960366,2016-03-28,13231,8.93,8.93,0.0,3.19,0.79,4.95,0.0,39,20,224,1080,1932
4,1503960366,2016-03-29,12041,7.85,7.85,0.0,2.16,1.09,4.61,0.0,28,28,243,763,1886




Dataframe 2:


,Id,Time,Value
0,2022484408,2016-04-01 07:54:00,93
1,2022484408,2016-04-01 07:54:05,91
2,2022484408,2016-04-01 07:54:10,96
3,2022484408,2016-04-01 07:54:15,98
4,2022484408,2016-04-01 07:54:20,100




Dataframe 3:


...



Dataframe 6:


,Id,date,value,logId
0,1503960366,2016-03-13 02:39:30,1,11114919637
1,1503960366,2016-03-13 02:40:30,1,11114919637
2,1503960366,2016-03-13 02:41:30,1,11114919637
3,1503960366,2016-03-13 02:42:30,1,11114919637
4,1503960366,2016-03-13 02:43:30,1,11114919637




Dataframe 7:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,2016-04-05 23:59:59,53.299999,117.506384,22.0,22.969999,True,1459900799000
1,1927972279,2016-04-10 18:33:26,129.600006,285.719105,NaN,46.169998,False,1460313206000
2,2347167796,2016-04-03 23:59:59,63.400002,139.773078,10.0,24.770000,True,1459727999000
3,2873212765,2016-04-06 23:59:59,56.700001,125.002104,NaN,21.450001,True,1459987199000
4,2873212765,2016-04-07 23:59:59,57.200001,126.104416,NaN,21.650000,True,1460073599000


## 3. Analyze 
Now that data has be loaded and preprocessed, it's time for analysis. 

**Guiding questions:**
* What suprises did you find in the data?
* What trends and relationships did you find in the data?

**Key Tasks**
* Aggregate data so it's useful and accessible
* Organize and format data
* Perform calculations

In [11]:
# start with daily_activities dataframe

daily_activity.head(2)

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,2016-03-25,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,2016-03-26,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154


In [12]:
# print(daily_activity['ActivityDate'].min())
display(daily_activity.sort_values('ActivityDate', ascending=True).head(10))


    

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
165,4020332650,2016-03-12,5543,3.97,3.97,0.0,0.00,0.00,3.96,0.01,0,0,254,757,2990
197,4057192912,2016-03-12,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0,0,0,1440,1777
166,4020332650,2016-03-13,3226,2.31,2.31,0.0,0.00,0.00,2.28,0.00,0,0,136,771,2480
198,4057192912,2016-03-13,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0,0,0,1440,1777
199,4057192912,2016-03-14,8433,6.23,6.23,0.0,2.45,0.33,3.44,0.00,30,7,135,1268,2453
167,4020332650,2016-03-14,3023,2.17,2.17,0.0,0.00,0.00,2.14,0.00,0,0,145,1005,2570
168,4020332650,2016-03-15,5906,4.23,4.23,0.0,0.00,0.16,4.04,0.00,0,10,215,874,3016
200,4057192912,2016-03-15,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0,0,0,1440,1776
169,4020332650,2016-03-16,12483,8.99,8.99,0.0,1.45,0.57,6.90,0.00,25,14,309,599,3830
201,4057192912,2016-03-16,2139,1.56,1.56,0.0,0.00,1.30,0.26,0.00,0,28,15,1397,1935


In [13]:
print(daily_activity['Id'].nunique()) # 35 unique ids
unique_ids = daily_activity.Id.unique()
print(type(unique_ids))
unique_ids = unique_ids.tolist()
print(type(unique_ids))
print(unique_ids)
display(daily_activity[daily_activity['Id'] == 2891001357].sort_values('ActivityDate', ascending=True)) 
display(daily_activity[daily_activity['Id'] == 6391747486].sort_values('ActivityDate', ascending=True))
# display(daily_activity[daily_activity.TotalSteps == 0].sort_values('ActivityDate', ascending=True).reset_index(drop=True))

35
<class 'numpy.ndarray'>
<class 'list'>
[1503960366, 1624580081, 1644430081, 1844505072, 1927972279, 2022484408, 2026352035, 2320127002, 2347167796, 2873212765, 2891001357, 3372868164, 3977333714, 4020332650, 4057192912, 4319703577, 4388161847, 4445114986, 4558609924, 4702921684, 5553957443, 5577150313, 6117666160, 6290855005, 6391747486, 6775888955, 6962181067, 7007744171, 7086361926, 8053475328, 8253242879, 8378563200, 8583815059, 8792009665, 8877689391]


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
135,2891001357,2016-03-29,0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0,0,0,1440,1920
136,2891001357,2016-03-30,0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0,0,0,1440,1920
137,2891001357,2016-03-31,0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0,0,0,1440,1920
138,2891001357,2016-04-01,0,0.00,0.0,4.828032,0.0,0.0,0.00,0.0,0,660,0,780,4562
139,2891001357,2016-04-02,4126,3.22,0.0,3.218688,0.0,0.0,3.22,0.0,0,0,720,720,2881
140,2891001357,2016-04-03,0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0,0,0,1440,1920
141,2891001357,2016-04-04,0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0,0,0,1440,1920
142,2891001357,2016-04-05,2063,1.61,0.0,1.609344,0.0,0.0,1.61,0.0,0,0,630,99,1144


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
334,6391747486,2016-04-01,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0,0,0,1440,1820
335,6391747486,2016-04-02,2101,2.03,0.00,2.027773,2.03,0.00,0.00,0.0,46,0,0,1394,2224
336,6391747486,2016-04-03,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0,0,0,1440,1820
337,6391747486,2016-04-04,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0,0,0,1440,1820
338,6391747486,2016-04-05,9766,7.51,0.78,6.727057,0.00,0.05,7.46,0.0,0,6,301,1133,2442
339,6391747486,2016-04-06,165,0.13,0.13,0.000000,0.00,0.00,0.13,0.0,0,0,8,1432,1849
340,6391747486,2016-04-07,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0,0,0,1440,1822
341,6391747486,2016-04-08,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0,0,0,1440,1820
342,6391747486,2016-04-09,0,0.00,0.00,0.000000,0.00,0.00,0.00,0.0,0,0,0,198,251


In [14]:
# calculate aggregates

# avg steps per user 
avg_steps_per_user = daily_activity.groupby('Id').TotalSteps.mean()
avg_steps_per_user = avg_steps_per_user.to_frame().sort_values('TotalSteps', ascending=True)
print(type(avg_steps_per_user))

print(avg_steps_per_user)

# user with total average steps = 8877689391

# sum of all steps per user
total_steps_per_user = daily_activity.groupby('Id').TotalSteps.sum()

total_steps_per_user = total_steps_per_user.to_frame().sort_values('TotalSteps', ascending=True)

print(total_steps_per_user)

<class 'pandas.core.frame.DataFrame'>
              TotalSteps
Id                      
2891001357    773.625000
1927972279   1269.069767
6391747486   1336.888889
4057192912   2103.972222
8792009665   2217.000000
1844505072   2876.023256
6775888955   3301.228571
4020332650   4049.761905
2320127002   4276.372093
6290855005   4615.846154
4445114986   4632.369565
8253242879   4898.064516
2026352035   4960.139535
1624580081   5167.200000
8583815059   6346.615385
3372868164   6616.933333
4558609924   7154.930233
2873212765   7299.255814
6117666160   7363.000000
4319703577   7422.813953
1644430081   7780.925000
4702921684   8367.065217
5577150313   8385.926829
7086361926   8459.813953
5553957443   8540.627907
8378563200   8555.162791
4388161847   8595.692308
2347167796   9647.121212
3977333714  10321.523810
6962181067  10679.888889
2022484408  11595.093023
7007744171  11619.289474
1503960366  11935.780000
8053475328  14784.523810
8877689391  16424.325581
            TotalSteps
Id            

In [15]:
# find out what days people took the most steps?
# pd.set_option('display.max_rows', None)
total_steps_by_day = daily_activity.groupby('ActivityDate').TotalSteps.sum()
total_steps_by_day = total_steps_by_day.to_frame().sort_values('TotalSteps', ascending=True)
print(total_steps_by_day) # 2016-04-12 had the most steps = 314095 -- the last day of the data

              TotalSteps
ActivityDate            
2016-03-18          1317
2016-03-24          1958
2016-03-13          3226
2016-03-12          5543
2016-03-19          5702
...                  ...
2016-04-06        263630
2016-04-21        263795
2016-04-23        267124
2016-04-16        277733
2016-04-12        314095

[62 rows x 1 columns]


In [16]:
sum_steps_calories_per_user = daily_activity.groupby('Id').agg(
    TotalCalories = ('Calories', 'sum'), 
    TotalSteps = ('TotalSteps', 'sum')
)

print(sum_steps_calories_per_user)
# calories vs. steps

            TotalCalories  TotalSteps
Id                                   
1503960366          90437      596789
1624580081          71689      258360
1644430081         113503      311237
1844505072          68169      123669
1927972279          94405       54570
2022484408         107513      498589
2026352035          64026      213286
2320127002          71834      183884
2347167796          67102      318355
2873212765          79775      313868
2891001357          18187        6189
3372868164          57265      198508
3977333714          62187      433504
4020332650         172372      255135
4057192912          68808       75743
4319703577          87099      319181
4388161847         110352      335232
4445114986          99389      213089
4558609924          84993      307662
4702921684         134254      384885
5553957443          79776      367247
5577150313         137092      343823
6117666160          84305      279794
6290855005          97045      180018
6391747486  